In [117]:
import json
import spacy
import yaml
import os
import operator
import copy
from array import array
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
def import_bios(path):
    with open(path) as fp:
        temp = json.load(fp)
        ids = [i['_id'] for i in temp['results'] if i['bio']]
        bio = [i['bio'] for i in temp['results'] if i['bio']]
        return ids,bio

In [138]:
ids,bios = import_bios('/home/pallavi/Project/SmartDate/output/Boston/2019-06-05.json')

In [139]:
df = pd.DataFrame(list(zip(ids, bios)), 
               columns =['ID', 'Bio']) 
df.head()

,ID,Bio
0,5cd61539b075641500e04dd3,Chipotle and fitness just about sums me up\n\n...
1,5c776a734794331100d2190c,I like big boats and I cannot lie
2,5c7ed51bdb31071100b47bef,Insta:ava_homsey
3,5c1b18ba6d5763eb149fd800,Future hockey mom\nvenmo- alyssa-Stcroix\nSnap...
4,5ce71c27f72e8a15002ebeb1,“Could be cooler” -VOGUE \n“If only she could...


In [140]:
train_texts = df['Bio'].values
train_texts = [s.lower() for s in train_texts]

In [141]:
train_texts[2]

'insta:ava_homsey'

In [142]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
vocab_size = len(alphabet)+1

char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1
tk.word_index = char_dict.copy()
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1

max_len = len(max(bios,key=len))
sequences = tk.texts_to_sequences(train_texts)
train_seq = pad_sequences(sequences, maxlen=max_len, padding='post')

In [143]:
X = copy.deepcopy(train_seq)
y = pad_sequences(train_seq[:,1:], maxlen=max_len, padding='post')

In [144]:
X = to_categorical(X)
y = to_categorical(y)
print(X.shape)
print(y.shape)

(69, 376, 38)
(69, 376, 38)


In [170]:
#LSTM
hidden_size = 100;
model = Sequential()
# model.add(Embedding(max_len, hidden_size, input_shape = X.shape)
model.add(LSTM(38,batch_size = None,input_shape = (X.shape[1],X.shape[2],), return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dense(38, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_44 (LSTM)               (None, 376, 38)           11704     
_________________________________________________________________
lstm_45 (LSTM)               (None, 376, 100)          55600     
_________________________________________________________________
lstm_46 (LSTM)               (None, 376, 100)          80400     
_________________________________________________________________
dense_8 (Dense)              (None, 376, 38)           3838      
Total params: 151,542
Trainable params: 151,542
Non-trainable params: 0
_________________________________________________________________
None


In [171]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=2)

Epoch 1/100
 - 4s - loss: 3.6026 - acc: 0.4147
Epoch 2/100
 - 2s - loss: 3.3751 - acc: 0.7971
Epoch 3/100
 - 2s - loss: 2.6462 - acc: 0.7971
Epoch 4/100
 - 2s - loss: 1.2336 - acc: 0.7971
Epoch 5/100
 - 2s - loss: 1.1095 - acc: 0.7971
Epoch 6/100
 - 2s - loss: 1.1972 - acc: 0.7971
Epoch 7/100
 - 2s - loss: 1.1792 - acc: 0.7971
Epoch 8/100
 - 2s - loss: 1.0752 - acc: 0.7971
Epoch 9/100
 - 2s - loss: 0.9035 - acc: 0.7958
Epoch 10/100
 - 2s - loss: 0.9322 - acc: 0.7840
Epoch 11/100
 - 2s - loss: 0.9170 - acc: 0.7847
Epoch 12/100
 - 2s - loss: 0.8213 - acc: 0.7951
Epoch 13/100
 - 2s - loss: 0.7950 - acc: 0.7971
Epoch 14/100
 - 2s - loss: 0.7595 - acc: 0.7969
Epoch 15/100
 - 2s - loss: 0.7337 - acc: 0.7969
Epoch 16/100
 - 2s - loss: 0.7215 - acc: 0.7976
Epoch 17/100
 - 2s - loss: 0.7067 - acc: 0.7973
Epoch 18/100
 - 2s - loss: 0.6997 - acc: 0.7974
Epoch 19/100
 - 2s - loss: 0.6930 - acc: 0.8001
Epoch 20/100
 - 2s - loss: 0.6868 - acc: 0.8149
Epoch 21/100
 - 2s - loss: 0.6813 - acc: 0.8247
E